In [1]:
!python -m pip install --upgrade pip

In [2]:
!pip install typing_extensions pydantic openai

In [3]:
!pip install datasets transformers peft trl bitsandbytes

In [4]:
import os
import torch
import json
import time
import random
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig, GenerationConfig, AutoModelForSequenceClassification
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import ORPOTrainer, ORPOConfig, AutoModelForCausalLMWithValueHead
from trl.trainer.utils import DPODataCollatorWithPadding
from tqdm import tqdm

c:\Users\ljh10\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

---

In [9]:
dataset_path = './korean_poetry_dataset.json'

with open(dataset_path, 'r', encoding='utf-8') as f:
    poem_data = json.load(f)

preprocessed_data = [{'topic': item['text']['topic'], 'poem': item['text']['poem']} for item in poem_data]

train_dataset = Dataset.from_list(preprocessed_data)


In [10]:
# Tokenizer 로드
model_name = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [12]:
# 데이터 전처리 함수 (토큰화 + labels 추가)
def preprocess_text(sample):
    input_texts = [f"주제: {t}\n시: {p}" for t, p in zip(sample['topic'], sample['poem'])] 

    model_inputs = tokenizer(input_texts, 
              padding='max_length',
              max_length=512,
              truncation=True
            )
    
    model_inputs['labels'] = model_inputs["input_ids"].copy()
    pad_token_id = tokenizer.pad_token_id
    model_inputs['labels'] = [
        [(l if l != pad_token_id else -100) for l in label] for label in model_inputs['labels']
    ]
    
    return model_inputs


In [13]:
# 데이터셋 변환
train_dataset = train_dataset.map(preprocess_text, batched=True, remove_columns=['topic','poem'])

Map: 100%|██████████| 2600/2600 [00:00<00:00, 3142.15 examples/s]


In [14]:
# 데이터 콜레이터
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

In [15]:
# VRAM 최적화를 위한 4-bit 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [16]:
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = 'auto'
)

c:\Users\ljh10\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ljh10\.cache\huggingface\hub\models--Bllossom--llama-3.2-Korean-Bllossom-3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers

RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

In [ ]:
# 양자화 모델 훈련을 위한 준비
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA 적용
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.train()   # 모델 학습 모드 설정

In [ ]:
# Trainer 설정
training_args = TrainingArguments(
    output_dir='./q_lora_poem',
    evaluation_strategy='no',
    save_strategy='epoch',
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    optim='adamw_bnb_8bit',
    report_to='none',
)

trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset= train_dataset,
    tokenizer = tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()